<div class="alert alert-block alert-success">
<b>NOTEBOOK 2 - Data Cleaning
</div>

---
# 1 - IMPORTS

### 1.1 - SETUP PROJECT

The same setup will be used in the all the noteboos so I am not going to include any more comments on that.

In [13]:
# IMPORTS

# Standard libraries
import sys
import importlib
from pathlib import Path
from typing import Union

# Third-party: core scientific
import numpy as np
import pandas as pd

# Add "../src/utilities" to sys.path for custom utilities
sys.path.append("../src/utilities")  # Ensure src/ is in path

# Import utils
try:
    import utils
    importlib.reload(utils)   # Ensures latest version is loaded
except ImportError as e:
    raise ImportError(f"Could not import utils module: {e}")

---
# 2 - DATASET LOAD

### 2.1 - LOADING

Here I have loaded the dataset back in.

In [14]:
# Load dataset
df = utils.load_dataset('../data/interim/01_students_dropout_data_forCleaning.xlsx')

---
# 3 - DATA LEAKAGE PREVENTION

### 3.1 - EARLY POTENTIAL LEAKAGE PREVENTION

To start off, I have used a few code lines to flag any suspiciously future-looking columns. It turned out no columns include any of these words "future|after|label".

In [15]:
### 3.5 - EARLY POTENTIAL LEAKAGE DETECTION

leak_test = df.columns[df.columns.str.contains("future|after|label")]

if len(leak_test) == 0:
    print("No potential leakage columns found.")    
else:
    print("Potential leaks:", leak_test.tolist())

No potential leakage columns found.


### 3.2 - PREDICTION HORIZON

*For this project, the chosen prediction horizon is after the first semester.*  
This means the model will only use features available *up to and including the end of Semester 1*, while excluding all features that become observable in Semester 2 or later.

**Concept**:
Predictions are made at a fixed *prediction time* (t₀), for an outcome that happens by *t₀ + h* (the horizon).  
All input features must be observed **on or before t₀**.  
- Using **future data (after t₀)** → *label leakage* (cheating).  
- Using **overly old data** → *noise*.

**Why this horizon?**

- *Early enough to intervene*  
  By the end of the first semester, students still have the majority of their academic program ahead of them.  
  This provides institutions with a realistic window to intervene through tutoring, mentoring, financial aid, or counseling.

- *Stronger predictive power*  
  Features such as 'grades', 'approved curricular units', and 'number of evaluations in Semester 1' capture students’ real engagement and performance.  
  These are well-established predictors of future dropout and academic success.

- *Balances actionability and accuracy*  
  - *Enrollment-only horizon*: highly proactive, but predictions are weak since no academic performance data is available yet.  
  - *Semester 2 horizon*: more accurate, but interventions may come too late, after many dropouts already occur.  
  - *Semester 1 horizon*: the *compromise point* — it provides meaningful signals while leaving enough time for support strategies.


**Caveats**

- *Not as early as enrollment*  
  Students who drop out within the first semester cannot be captured by this approach.  
  Therefore, the model is designed to prevent *later* dropouts, not immediate ones.

- *Risk of label leakage*  
  It is essential to strictly exclude all Semester 2 features ('grades', 'approved units', 'evaluations', etc.), since they would not be available at the chosen horizon. Using them would artificially inflate model performance and make it unrealistic in practice.  
  Also, it is necessary to remove all of the students who dropped during the 1st semester without any evaluation data, because they were no longer observable at decision time. Including them would introduce data leakage.


**Target vs Features**

- The target ('Target' column) is not decided at Semester 1 – it is decided later, after years of observation (official end of the course).  
- At the time of collection, there are three different label classes:  
  - *'Dropout'* → the student left the program (at or after Semester 1).  
  - *'Enrolled'* → the student was still studying when the dataset snapshot ended.  
  - *'Graduate'* → the student finished the degree.  

*Key point:* The horizon only limits which *features* can be used. The *target* is always defined in the future, and that is what the model is trying to predict.


**Population groups considered**

Since the time horizon is fixed at the end of Semester 1, I will keep:  
- All demographics / socio-economic / administrative info, established at the beginning of the program.  
- All students who dropped out *after completing the first semester*, because they are observable at the prediction point.  
- All students who remained enrolled or graduated, since their status is decided later and must be predicted.  

**I will drop:**  
- Any feature collected after Semester 1 (to avoid peeking into the future).  
- All students who dropped before completing the first semester, since they were not observable at decision time (they had already dropped).


**Conclusion**

The *Semester 1 horizon* provides a justified compromise between *early detection* and *model reliability*.  
It allows institutions to identify at-risk students with meaningful accuracy, while keeping predictions realistic and actionable within the academic cycle.


In [16]:
# DELETE "AT RISK" COLUMNS

# Remove any columns related to 2nd semester
for col in df.columns:
    if '2nd' in col:
        df.drop(columns=[col], inplace=True)

# Remove "Target" values that are impossible to know at the end of semester 1
sem1_activity_cols = []
for col in df.columns:
    if '1st sem' in col:
        sem1_activity_cols.append(col)

# Evidence of semester-1 activity = sum of values across these cols
evidence = df[sem1_activity_cols].fillna(0).sum(axis=1)
print(evidence)


# Exclude "Dropout" cases with zero sem1 evidence (likely dropped before end of 1st sem)
mask_valid = ~((df["Target"] == "Dropout") & (evidence == 0))

df_valid = df[mask_valid].copy()
print(f"Kept {df_valid.shape[0]} of {df.shape[0]} students")

df_valid.head()

0        0.000000
1       32.000000
2        6.000000
3       33.428571
4       32.333333
          ...    
4419    31.600000
4420    30.000000
4421    36.912500
4422    28.800000
4423    31.666667
Length: 4424, dtype: float64
Kept 4347 of 4424 students


,Marital Status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,14.000000,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,8,6,13.428571,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,9,5,12.333333,0,13.9,-0.3,0.79,Graduate
5,2,39,1,9991,0,19,133.1,1,37,37,...,0,5,10,5,11.857143,0,16.2,0.3,-0.92,Graduate


---
# 4 - STRUCTURAL CLEANUP

Here I am performing a **structural cleanup** which handles data without risking any leakage, as it is non-statistical. This allowes me to operate on the whole dataset (including the target) without making any distinction between train and test sets yet.

### 4.1 - COLUMNS FORMATTING AND RENAME

It's good practice to normalize column names and to keep them short and meaningful. This includes:
- lowercasing
- replacing spaces, slashes, tabs, and parentheses with _

In [17]:
# FORMATTING

print(df_valid.columns)
# Automatic renaming
df_valid.columns = (
    df_valid.columns
    .str.strip()              # remove accidental whitespace
    .str.lower()              # lowercase
    .str.replace(r"[^\w\s]", "", regex=True)  # remove special chars
    .str.replace(r"\s+", "_", regex=True)     # replace spaces with "_"
)

# Manual renaming (if needed)
df_valid = df_valid.rename(columns={
    "marital_status": "marital",
    "application_mode": "app_mode",
    "application_order": "app_order",
    "course": "course",
    "daytimeevening_attendance": "att_mode",
    "previous_qualification": "prev_qual",
    "previous_qualification_grade": "prev_grade",
    "nacionality": "nationality",
    "mothers_qualification": "mother_qual",
    "fathers_qualification": "father_qual",
    "curricular_units_1st_sem_credited": "sem1_credited",
    "curricular_units_1st_sem_enrolled": "sem1_enrolled",
    "curricular_units_1st_sem_evaluations": "sem1_evals",
    "curricular_units_1st_sem_approved": "sem1_approved",
    "curricular_units_1st_sem_grade": "sem1_grade",
    "curricular_units_1st_sem_without_evaluations": "sem1_noevals",
    "unemployment_rate": "unemp_rate",
    "inflation_rate": "inflation",
    "gdp": "gdp"
})

df_valid.head()

Index(['Marital Status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)', 'Unemployment rate',
       'Inflation rate', 'GDP', 'Target'],
      dtype='object')


,marital,app_mode,app_order,course,att_mode,prev_qual,prev_grade,nationality,mother_qual,father_qual,...,sem1_credited,sem1_enrolled,sem1_evals,sem1_approved,sem1_grade,sem1_noevals,unemp_rate,inflation,gdp,target
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,14.000000,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,8,6,13.428571,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,9,5,12.333333,0,13.9,-0.3,0.79,Graduate
5,2,39,1,9991,0,19,133.1,1,37,37,...,0,5,10,5,11.857143,0,16.2,0.3,-0.92,Graduate


I have devised a function (see cleaning.data_quality_check()) that performed an overall quality check on the dataset. In my case, there are no date/time or free-text features so Ijust needed to check for formatting issues or other inconsistencies. 

### 4.2 - FIX TYPES

For clarity and easier handling in EDA and plotting, I divided the features into **two broad groups**:

- **Categorical** → includes both *multi-class categories* (e.g., `marital`, `course`, `prev_qual`) and *binary variables* (e.g., `gender`, `debtor`, `scholarship_holder`).  
  Treating binary as categorical ensures they are grouped consistently when generating plots and summaries.

- **Numeric (int / float)** → continuous or discrete measures such as grades, age, semester credits, and macroeconomic indicators (`admission_grade`, `prev_grade`, `sem1_grade`, `unemp_rate`, `gdp`, etc.).

> This grouping simplifies exploratory analysis: categorical plots (bar charts, countplots) can be automated across all non-numeric features, while numeric plots (histograms, boxplots, density plots) are handled separately.

In [ ]:
# Check data types
utils.detect_variable_types(df_valid, target_col="target")
df_valid.dtypes

marital                        int64
app_mode                       int64
app_order                      int64
course                         int64
att_mode                       int64
prev_qual                      int64
prev_grade                   float64
nationality                    int64
mother_qual                    int64
father_qual                    int64
mothers_occupation             int64
fathers_occupation             int64
admission_grade              float64
displaced                      int64
educational_special_needs      int64
debtor                         int64
tuition_fees_up_to_date        int64
gender                         int64
scholarship_holder             int64
age_at_enrollment              int64
international                  int64
sem1_credited                  int64
sem1_enrolled                  int64
sem1_evals                     int64
sem1_approved                  int64
sem1_grade                   float64
sem1_noevals                   int64
u

In [19]:
CATEGORICAL_COLS = [
    "marital",
    "app_mode",
    "course",
    "att_mode",
    "prev_qual",
    "nationality",
    "mother_qual",
    "father_qual",
    "gender",
    "scholarship_holder",
    "displaced",
    "educational_special_needs",
    "debtor",
    "tuition_fees_up_to_date",
    "international",
]

for col in CATEGORICAL_COLS:
    if col in df_valid.columns:
        df_valid[col] = df_valid[col].astype("category")

df_valid.dtypes

marital                      category
app_mode                     category
app_order                       int64
course                       category
att_mode                     category
prev_qual                    category
prev_grade                    float64
nationality                  category
mother_qual                  category
father_qual                  category
mothers_occupation              int64
fathers_occupation              int64
admission_grade               float64
displaced                    category
educational_special_needs    category
debtor                       category
tuition_fees_up_to_date      category
gender                       category
scholarship_holder           category
age_at_enrollment               int64
international                category
sem1_credited                   int64
sem1_enrolled                   int64
sem1_evals                      int64
sem1_approved                   int64
sem1_grade                    float64
sem1_noevals

### 4.3 - RANGE CHECK (NUMERICAL)

In [20]:
# Configure expected values
EXPECTED_CATS = {
    "marital": {1, 2, 3, 4, 5, 6},
    "app_mode": {1, 2, 5, 7, 10, 15, 16, 17, 18, 26, 27, 39, 42, 43, 44, 51, 53, 57},
    "course": {33, 171, 8014, 9003, 9070, 9085, 9119, 9130, 9147, 9238, 9254, 9500, 9556, 9670, 9773, 9853, 9991},
    "att_mode": {0, 1},
    "prev_qual": {1, 2, 3, 4, 5, 6, 9, 10, 12, 14, 15, 19, 38, 39, 40, 42, 43},
    "nationality": {1, 2, 6, 11, 13, 14, 17, 21, 22, 24, 25, 26, 32, 41, 62, 100, 101, 103, 105, 108, 109},
    "mother_qual": {1,2,3,4,5,6,9,10,11,12,14,18,19,22,26,27,29,30,34,35,36,37,38,39,40,41,42,43,44},
    "father_qual": {1,2,3,4,5,6,9,10,11,12,13,14,18,19,20,22,25,26,27,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44},
    "mother_occupation": {0,1,2,3,4,5,6,7,8,9,10,90,99,122,123,125,131,132,134,141,143,144,151,152,153,171,173,175,191,192,193,194},
    "father_occupation": {0,1,2,3,4,5,6,7,8,9,10,90,99,101,102,103,112,114,121,122,123,124,131,132,134,135,141,143,144,151,152,153,154,161,163,171,172,174,175,181,182,183,192,193,194,195},
    "displaced": {0,1},
    "educational_special_needs": {0,1},
    "debtor": {0,1},
    "tuition_fees_up_to_date": {0,1},
    "gender": {0,1},
    "scholarship_holder": {0,1},
    "international": {0,1},
    "target": {"Dropout","Enrolled","Graduate"},
}

EXPECTED_RANGES = {
    "app_order": (0, 9),
    "prev_grade": (0, 200),
    "admission_grade": (0, 200),
    "sem1_grade": (0.0, 20.0),
    "sem2_grade": (0.0, 20.0),
    "unemp_rate": (0.0, 100.0),
    "inflation": (-50.0, 200.0),
    "gdp": (-10.0, 10.0),
    # non-negative counts
    "sem1_credited": (0, np.inf),
    "sem1_enrolled": (0, np.inf),
    "sem1_evals": (0, np.inf),
    "sem1_approved": (0, np.inf),
    "sem1_noevals": (0, np.inf),
    "age_at_enrollment": (0, np.inf),
}


# Ordered checklist output
print("# Categorical checks")
i = 1
for col in sorted(EXPECTED_CATS):
    if col not in df_valid.columns:
        print(f"{i}. {col}: not in dataframe")
        i += 1
        continue

    allowed = EXPECTED_CATS[col]
    s = df_valid[col]

    # coerce numeric-like codes
    if all(isinstance(v, (int, float)) for v in allowed):
        s = pd.to_numeric(s, errors="coerce").astype("Int64")
    else:
        s = s.astype("string")

    unexpected = set(s.dropna().unique()) - set(allowed)
    if len(unexpected) == 0:
        print(f"{i}. {col}: OK — all values within allowed set")
    else:
        print(f"{i}. {col}: ISSUE — unexpected values: {sorted(list(unexpected))[:10]}")
    i += 1

print("\n# Numeric checks")
j = 1
for col in sorted(EXPECTED_RANGES):
    if col not in df_valid.columns:
        print(f"{j}. {col}: not in dataframe")
        j += 1
        continue

    lo, hi = EXPECTED_RANGES[col]
    s = pd.to_numeric(df_valid[col], errors="coerce")
    bad = s.notna() & ((s < lo) | (s > hi))

    if bad.any():
        print(f"{j}. {col}: ISSUE — {int(bad.sum())} out of range; "
              f"observed min={s.min(skipna=True):.3g}, max={s.max(skipna=True):.3g}, "
              f"expected {lo}..{hi}")
    else:
        print(f"{j}. {col}: OK — all values within [{lo}, {hi}]")
    j += 1

df_valid.dtypes

# Categorical checks
1. app_mode: OK — all values within allowed set
2. att_mode: OK — all values within allowed set
3. course: OK — all values within allowed set
4. debtor: OK — all values within allowed set
5. displaced: OK — all values within allowed set
6. educational_special_needs: OK — all values within allowed set
7. father_occupation: not in dataframe
8. father_qual: OK — all values within allowed set
9. gender: OK — all values within allowed set
10. international: OK — all values within allowed set
11. marital: OK — all values within allowed set
12. mother_occupation: not in dataframe
13. mother_qual: OK — all values within allowed set
14. nationality: OK — all values within allowed set
15. prev_qual: OK — all values within allowed set
16. scholarship_holder: OK — all values within allowed set
17. target: OK — all values within allowed set
18. tuition_fees_up_to_date: OK — all values within allowed set

# Numeric checks
1. admission_grade: OK — all values within [0, 200]
2. ag

marital                      category
app_mode                     category
app_order                       int64
course                       category
att_mode                     category
prev_qual                    category
prev_grade                    float64
nationality                  category
mother_qual                  category
father_qual                  category
mothers_occupation              int64
fathers_occupation              int64
admission_grade               float64
displaced                    category
educational_special_needs    category
debtor                       category
tuition_fees_up_to_date      category
gender                       category
scholarship_holder           category
age_at_enrollment               int64
international                category
sem1_credited                   int64
sem1_enrolled                   int64
sem1_evals                      int64
sem1_approved                   int64
sem1_grade                    float64
sem1_noevals

**Structural Cleanup Summary**

During the **structural cleanup phase**, the dataset was reviewed for common quality issues.  
All of the following checks were performed:

- **Duplicates** → none found.  
- **Obvious errors / impossible values** → none detected.  
- **Encoding errors** → not present.  
- **Wrong units** → no inconsistencies identified.  
- **Format inconsistencies** → none observed across columns.  
- **Future dates / date issues** → not applicable (no datetime fields).  
- **Text standardization** → not needed (no free-text fields).  
- **Datetime formatting** → not applicable.  
- **Hard clipping of ranges** → not required, since features had already been preprocessed and values fell within plausible bounds.

**Data Quality Scorecard**

| Metric             | Excellent      | Good         | Needs Attention | Critical Issue |
|--------------------|----------------|--------------|-----------------|----------------|
| **Missing Values** | <1%            | 1–5%         | 5–15%           | >15%           |
| **Duplicate Records** | 0%          | <0.5%        | 0.5–2%          | >2%            |
| **Invalid Values** | 0%             | <0.1%        | 0.1–1%          | >1%            |
| **Outliers**       | <2%            | 2–5%         | 5–10%           | >10%           |


**Current Dataset Assessment**

- **Missing Values** → **0%** → Excellent  
- **Duplicate Records** → **0%** → Excellent  
- **Invalid Values** → **0%** → Excellent  
- **Outliers** → *to be evaluated on the train set (EDA step)*  


The dataset was provided by UCI and underwent **rigorous preprocessing** before release.  
As a result:
>- **Missing values** are absent,  
>- **Duplicate records** were not detected,  
>- **Invalid or inconsistent values** were not present after structural cleanup.  
>- These checks confirm that the dataset did not require **structural corrections** for these categories.  
>- Most potential issues were either **addressed during prior preprocessing** or are **not applicable** (e.g., absence of text or date fields). This will help improve model performance and decision-making.
>- The only remaining potential quality issue is **outliers**, which will be investigated during the **EDA phase** on the **training set only**, to avoid data leakage.  
This ensures that anomaly detection and mitigation strategies are **aligned with the actual modeling data**.  
>- The dataset is therefore ready to be **split** and to move on to the **analysis and exploratory phase**.

Overall, the dataset health is **high** and suitable for moving forward to preprocessing and modeling.

In [22]:
# Save dataset as Excel for manual inspection
utils.save_dataset(df_valid, 'interim/02_dataset_structural_cleanup.xlsx')

# Save dataset as pickle for faster loading later (+ keeps dtypes)
df_valid.to_pickle("../data/interim/02_dataset_structural_cleanup.pkl")

File saved at: C:\Users\Vaccari\Desktop\iCloudDrive\Desktop\ENRICO\05_LEARNING\University\ToU\Phases\02_Calibration_Phase\Applied_Machine_Learning\Classification\Early_Identification_Of_At-Risk_Students\data\interim\02_dataset_structural_cleanup.xlsx


<div class="alert alert-block alert-info">
<b>Next Notebook - Data Splitting
</div>